In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
#数据导入
data=pd.read_csv('/kaggle/input/mnist-digit-recognizer/train.csv')
#输出前五行数据
data.head()


In [ ]:
data = np.array(data)
m, n = data.shape

#shuffle:将所有数据随机排序
np.random.shuffle(data)
#读入前1000行数据，并转置
data_dev = data[0:1000].T 
#将真实结果存入
Y_dev = data_dev[0]
#训练数据初始化
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape


In [ ]:
m,n,_,m_train,Y_dev.shape

## Neural Network

我们的神经网络将有一个简单的两层架构。
输入层a[0]将具有784个单元，对应于每个28x28输入图像中的784像素。
一个隐藏层A[1]将有10个单元进行ReLU激活
最后我们的输出层A[2]将有10个单元对应于10个数字类进行softmax激活。

## Vars&shapes

### Forward prop

* $A^{[0]}=X : 784 \times \space m $
* $Z^{[1]}∼A^{[1]} : 10 \times \space m$
* $W^{[1]} : 10 \times 784 (as \space W^{[1]}A^{[0]}∼Z^{[1]})$
* $B^{[1]} : 10 \times 1$
* $Z^{[2]}∼A^{[2]} : 10 \times \space m$
* $W^{[1]} : 10 \times 10 (as \space W^{[2]}A^{[1]}∼Z^{[2]} )$
* $B^{[2]} : 10 \times 1$

### Backword prop

* $dZ^{[2]} : 10 \times \space m(A^{[2]})$
* $dW^{[2]} : 10 \times \space 10$
* $dB^{[2]} : 10 \times \space1$
* $dZ^{[1]} : 10 \times \space m(A^{[1]})$
* $dW^{[1]} : 10 \times \space 10$
* $dB^{[1]} : 10 \times \space 1$

## Formula

### Forward propagation(正向传播)


$$Z^{[1]}=W^{[1]}X+b^{[1]}$$
 
$$A^{[1]}=gReLU(Z^{[1]}))$$
 
$$Z^{[2]}=W^{[2]}A^{[1]}+b^{[2]}$$
 
$$A^{[2]}=gsoftmax(Z^{[2]})$$
 

### Backward propagation(反向传播)
$$dZ^{[2]}=A^{[2]}-Y$$
 
$$dW^{[2]}=\frac 1 m dZ^{[2]}A^{[1]T}$$
 
$$dB^{[2]}=\frac 1 m \sum dZ^{[2]}$$
 
$$dZ^{[1]}=W^{[2]T}dZ^{[2]}\cdot g^{[1]}{'}(z^{[1]})$$
 
$$dW^{[1]}=\frac 1 m dZ^{[1]}A^{[0]T}$$
 
$$dB^{[1]}=\frac 1 m \sum dZ[1]$$
 
### Parameter updates(参数更新)

$$W^{[2]}:=W^{[2]}− \alpha dW^{[2]}$$
 
$$b^{[2]}:=b^{[2]}−\alpha db^{[2]}$$
 
$$W^{[1]}:=W^{[1]}−\alpha dW^{[1]}$$
 
$$b^{[1]}:=b^{[1]}−\alpha db^{[1]}$$
 


In [ ]:
def init_params():
    W1=np.random.rand(10,784)-0.5
    b1=np.random.rand(10,1)-0.5
    W2=np.random.rand(10,10)-0.5
    b2=np.random.rand(10,1)-0.5
    return W1, b1, W2, b2
def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [ ]:
def get_predictions(A2):
    #np.argmax(input.axis)
    #axis=0:计算每列的最大值，并输出其序号
    #axis=1:计算每行的最大值，并输出其序号
    return np.argmax(A2, 0)
#计算准确率
def get_accuracy(predictions, Y):
    print('预测结果:')
    print(predictions)
    print('真实结果：')
    print(Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i%10==0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
for i in range(10):
    test_prediction(int(np.random.randint(0,1000)), W1, b1, W2, b2)

In [ ]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
print('准确率:'+str(get_accuracy(dev_predictions, Y_dev)))